In [1]:
import sys
import pandas as pd
from thesaurus import Word
from nltk.tokenize import word_tokenize
from nltk import tokenize
from random import randint, shuffle
import nltk.data
import thesaurus as th
import ipdb
from datetime import datetime
import itertools
from collections import Iterable

import spacy as nlp

In [2]:
df = pd.read_csv('/opt/flask/src/Flask_Core/Nltk/Data/Datasets/datathon_v2_trainable.csv')
df.head()

,Sample,Label
0,I fell off my board at full speed and broke my...,Injury/Health Issue
1,I wiped out and was injured,Injury/Health Issue
2,My board failed on me mid-ride and I suffered ...,Injury/Health Issue
3,I've experienced bodily harm as a result of yo...,Injury/Health Issue
4,Being around the board when it is active makes...,Injury/Health Issue


In [3]:
len(df)

2122

In [4]:
syn_limit = 10
perm_limit = 100
random_perm = True

In [7]:
sp_model = nlp.load('en')
def synonym_scrambling(sentence, synonym_limit=None, permutation_limit=None, random_permutations=False):
    words = sp_model(sentence)
    synonym_dict = {}
    for word in words:
        # Do not process stop words or relevant POS
        synonym = None
        if word.is_stop or word.pos_ in ['PROPN', 'SYM'] or word.text.lower() in ['board', 'hoverboard', 'comic', 'issue', 'huva']:
            pass
        else:
            try:
                synonym = th.Word(word.text).synonyms(relevance=3)
                synonym = synonym[:synonym_limit] if synonym_limit and len(synonym) > synonym_limit else synonym
                if synonym != []: 
                    synonym_dict[word.text] = synonym
                else:
                    pass
            except:
                pass
            
        synonym_dict[word.text] = synonym
            
    syn_list=[]
    for key, value in synonym_dict.items():
        temp = [key, value]
        syn_list.append(temp)
        
    synonymlist = []
    for i in range(len(syn_list)):
        flat = flattern(syn_list[i])
        synonymlist.append(flat)
        
    matrix_combine = remove_none(synonymlist)
    #ipdb.set_trace()
    answer = [' '.join(perm) for perm in itertools.islice(itertools.product(*matrix_combine), 10)]
    if random_permutations:
        shuffle(answer)
    if permutation_limit:
        answer = answer[:permutation_limit]
    
    answer.append(sentence)
    answer = list(set(answer))
    
    return answer
    
def flattern(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flattern(i))
        else: rt.append(i)
    return rt

def remove_none(flattenlist):
    none_removed = []
    for i in range(len(flattenlist)):
        remove_none = list(filter(None, flattenlist[i]))
        none_removed.append(remove_none)
    return none_removed

def make_dataframe(samples):
    df_output = pd.DataFrame(samples, columns=['Sample'])
    return df_output

def make_labellist(label, count):
    label_list = []
    for j in range(count):
        labellst = (label)
        label_list.append(labellst)
    return label_list

In [8]:
comb = []
num_samples = len(df['Sample'])
for i, sentence in enumerate(df['Sample']):
    comb.append(synonym_scrambling(sentence, synonym_limit=syn_limit, permutation_limit=perm_limit, random_permutations=random_perm))
    sys.stdout.write("\r" + f"Processed {i+1}/{num_samples} Sample")
    sys.stdout.flush()

label = [make_labellist(label, len(synonym_set)) for synonym_set, label in zip(comb, df['Label'])]

Processed 2122/2122 Sample

In [9]:
flattened_list = [y for x in comb for y in x]
flattened_label_list = [y for x in label for y in x]

# ipdb.set_trace()

output_df = make_dataframe(flattened_list)
output_df['Label'] = flattened_label_list
output_df.head()

,Sample,Label
0,I fell off my board at full speed and broke leg,Injury/Health Issue
1,I fell off my board at full speed and bankrupt...,Injury/Health Issue
2,I fell off my board at full speed and bankrupt...,Injury/Health Issue
3,I fell off my board at full speed and bankrupt...,Injury/Health Issue
4,I fell off my board at full speed and broke pole,Injury/Health Issue


In [10]:
output_df.to_csv(path_or_buf='/opt/flask/src/Flask_Core/Nltk/Data/Datasets/datathon_v2_trainable_10_10_updated.csv', index=False)

In [11]:
len(output_df)

21427